In [10]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import scipy.stats as st
import sympy as sp
import scipy.integrate as si
import math

In [2]:
company_dataset = pd.read_csv("Data/CRSP_stocks_clean.csv")

/var/folders/9x/zgzyj9l153v5qjjz_hdvd31r0000gn/T/ipykernel_36701/877988600.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  company_dataset = pd.read_csv("Data/CRSP_stocks_clean.csv")


In [3]:
company_dataset[company_dataset["TICKER"]=="AA"]

,Unnamed: 0,date,TICKER,CUSIP,PRC,RET,VOL,BID,ASK,SHROUT
636697,887,2000-01-03,AA,44320110,80.9375,-0.024849,1551299.0,80.750,81.125,366407.0
636698,9421,2000-01-04,AA,44320110,81.3125,0.004633,2234799.0,81.125,81.500,366407.0
636699,17954,2000-01-05,AA,44320110,86.0000,0.057648,3121599.0,85.750,86.250,366407.0
636700,26490,2000-01-06,AA,44320110,84.8750,-0.013081,4494699.0,84.625,85.125,366407.0
636701,35023,2000-01-07,AA,44320110,84.6250,-0.002946,4534699.0,85.000,85.375,366407.0
...,...,...,...,...,...,...,...,...,...,...
640927,30094608,2016-10-25,AA,44320110,27.2900,0.016009,9682861.0,27.290,27.300,438460.0
640928,30101849,2016-10-26,AA,44320110,27.7900,0.018322,7794174.0,27.790,27.800,438460.0
640929,30109093,2016-10-27,AA,44320110,28.2200,0.015473,9271414.0,28.210,28.220,438460.0
640930,30116338,2016-10-28,AA,44320110,28.3700,0.005315,8937600.0,28.360,28.370,438460.0


In [68]:
#to change when we have more recent data
today = pd.to_datetime(date(2022, 12, 30))
#create historical mean starting date, should be about 3 years from current
historicalstart = (pd.to_datetime(date(2019,12,30)))
#pre-set lambda value
Lambda = 0.97
alpha = 0.01

AttributeError: 'Timestamp' object has no attribute 'tostring'

In [5]:
testdf = company_dataset
testdf = testdf[testdf['TICKER']=='AAPL']
print(testdf.dtypes)

Unnamed: 0      int64
date           object
TICKER         object
CUSIP          object
PRC           float64
RET           float64
VOL           float64
BID           float64
ASK           float64
SHROUT        float64
dtype: object


In [21]:
def ConditionalMean(returns,startdate):
    #filter rows where date is after startdate
    conditionalMeanDF = returns[returns['date'] > startdate]
    #take the mean return
    conditionalmean = conditionalMeanDF["RET"].mean()
    return conditionalmean

def VolatilityDF(returns):
   #Calculate standard deviation of returns
    volatilitydf = returns
    volatilitydf.set_index('date',inplace=True)
    volatilitydf["Variance"] = volatilitydf.rolling(window=22).var()
    volatilitydf["Std"] = np.sqrt(volatilitydf["Variance"])
    volatilitydf = volatilitydf.reset_index()
    return(volatilitydf)

def ConditionalVariance(volatilitydf,today):
    yesterday = volatilitydf[["date"] == (today- datetime.timedelta(days=1)).date()]
    today = volatilitydf[["date"] == today]
    yesterdayReturn = yesterday.iloc[0]["RET"]
    yesterdayVariance = yesterday.iloc[0]["Variance"]
    conditionalvariance = sqrt((Lambda*yesterdayVariance)+ (1-Lambda)*yesterdayReturn)
    return conditionalvariance
def ExpZScore(alpha):
    pi = math.pi
    return (((1 / math.sqrt(2*pi)) * math.exp(-x**2/2))*x/0.01)
    lower_limit = -sp.oo  # Negative infinity
    upper_limit = st.norm.ppf(alpha)
    expectedZscore = si.quad(ZScore,lower_limit, upper_limit)[0]
    return(expectedZscore)  

In [69]:
# groupy by ticker
grouped_data = company_dataset.groupby('TICKER')
returns = pd.DataFrame({})
for ticker,data in grouped_data:
    #creating a returns dataframe for each ticker with return and date columns, then indexing by date
    returns = data[['RET','date']].dropna()
    returns['date'] = pd.to_datetime(returns['date'])
    if (today in returns['date']):
        returns.set_index('date',inplace=True)
        returns = returns.rolling(window=22).mean()
        returns = returns.reset_index()
        condmean = ConditionalMean(returns,historicalstart)
        volatilitydf = VolatilityDF(returns)
        conditionalvariance = ConditionalVariance(volatilitydf,today)
        expzscore = ExpZScore(alpha)
        print(expzscore, conditionalvariance, volatilitydf, condmean)
        expectedshortfall = condmean + conditionalvariance * expzscore
        print(expectedshortfall)
    else:
        pass

In [20]:
(today - datetime.timedelta(1)).date()

datetime.date(2022, 12, 29)

In [67]:
print(company_dataset['date'][0])
jj = pd.to_datetime(company_dataset['date']).dt.date
jj = pd.to_datetime(jj)
jj
print(type(jj[1]))
print(type((historicalstart)))
historicalstart in jj

2000-01-03
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'numpy.datetime64'>


False